### Word Group In Youtube

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 1000  # native word end index 28,200,1000,5000,10000, 20000, 40000

# word ratio
if word_end == 28 or word_end == 200:
    unique_word_ratio = 70
    include_wordgroup_ratio = 70
elif word_end == 1000 or word_end == 5000:
    unique_word_ratio = 80
    include_wordgroup_ratio = 80
elif word_end == 10000:
    unique_word_ratio = 85
    include_wordgroup_ratio = 85
else:
    unique_word_ratio = 90
    include_wordgroup_ratio = 90

# word usage
if word_end == 28 or word_end == 200 or word_end == 1000 or word_end == 5000 or word_end == 10000:
    word_usage_min = 10
    word_usage_max = 100
elif word_end == 20000:
    word_usage_min = 7
    word_usage_max = 100
else:
    word_usage_min = 5
    word_usage_max = 100

# use for manuel set
#unique_word_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 ; 10000 = 85 ; 20000,40000 = 90
#include_wordgroup_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 : 10000 = 85 ; 20000,40000 = 90
#word_usage_min = 10  # 28,200,1000,5000,10000 = 10 ; 20000 = 7 ; 40000 = 5
#word_usage_max = 100


# sentence ratio and time shift
adjust_sent_word_ratio = 15  #28,200 = 15 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
#adjust_text_word_ratio = 5  #28,200 = 5 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
shift = 0.3  # sentence time shift

# func select
if word_end == 28:
    sentence_word_count_number = 2
else:
    sentence_word_count_number = 5   
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
                                  # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"word_end:{word_end}\nfile_ext:{file_ext}\nunique_word_ratio:{unique_word_ratio}\n\
include_wordgroup_ratio:{include_wordgroup_ratio}\nword_usage_min:{word_usage_min}\nword_usage_max:{word_usage_max}")

word_end:1000
file_ext:6
unique_word_ratio:80
include_wordgroup_ratio:80
word_usage_min:10
word_usage_max:100


In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Result/3-Word Group In Youtube"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by=f"{target_opt_column}", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [6]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []
    # check words in word_list
    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass
    # add index plus 1 and compare with current and next index (Is it continuous or not?)
    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []
    # index convert to word
    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [7]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        # check words in word_list
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass
        # add index plus 1 and compare with current and next index (Is it continuous or not?)
        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []
        # index convert to word
        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
        # dataframe output     
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [8]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [9]:
def repetation_ratio(text):
    '''
    repetation_ratio(text) 
    text is a like as sentence and def compute repetation word ratio\n
    repetation_ratio("yok bir şey yok bir şey yok bir şey") 
    '''
    word_list = word_tokenize(text)
    word_num = len(word_list)
    word_unique_num = len(set(word_list))
    ratio = int((word_unique_num/word_num)*100)
    
    return ratio

In [10]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all  # ?????

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987921,karneleme,5
987922,karnaya,5
987923,dörtlümüzün,5
987924,karnavalınız,5


In [11]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [12]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
995,hale,70625
996,kavga,70611
997,arabayı,70310
998,oldun,70121


In [13]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [14]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [15]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

1000

#### Youtube Sentence Analysis

In [16]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [17]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [18]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU
3,3,10.267,11.394,mumu,fI1BI4d5KFU
4,4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...,...
3934202,3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [19]:
#df_youtube_sentence[df_youtube_sentence["index"] == 1287503]

In [20]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/(len(sent_word)+0.001))*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [21]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU,28.6,"staff, dizinin, betimlemesi, fi, lm","tarafından, bu"
1,1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU,0.0,"sesli, yaptırılmıştır, betimleme, derneğine",
2,2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU,0.0,wwwsebederorg,
3,3,10.267,11.394,mumu,fI1BI4d5KFU,0.0,mumu,
4,4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU,42.9,"hat, sinir, katsayım, safhada","an, şu, bak"
...,...,...,...,...,...,...,...,...
3934202,3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0,43.7,"arazilerin, videoyu, danimarka, bitirmeden, yo...","bir, isterim, da, şunu, ki, olduğu, söylemek"
3934203,3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0,30.0,"bisikletle, yakını, ulaşımda, işine, yarısına,...","gidiyor, insanların, bu"
3934204,3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0,33.3,"söyleyeceklerim, alakalı, danimarkayla, şimdilik","kadar, bu"
3934205,3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0,11.1,"beğenmeyi, unutmayın, kanalıma, hoşunuza, vide...",ve


In [22]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU,28.6,"staff, dizinin, betimlemesi, fi, lm","tarafından, bu"
1,1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU,0.0,"sesli, yaptırılmıştır, betimleme, derneğine",NaN
2,2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU,0.0,wwwsebederorg,NaN
3,3,10.267,11.394,mumu,fI1BI4d5KFU,0.0,mumu,NaN
4,4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU,42.9,"hat, sinir, katsayım, safhada","an, şu, bak"
...,...,...,...,...,...,...,...,...
3934202,3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0,43.7,"arazilerin, videoyu, danimarka, bitirmeden, yo...","bir, isterim, da, şunu, ki, olduğu, söylemek"
3934203,3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0,30.0,"bisikletle, yakını, ulaşımda, işine, yarısına,...","gidiyor, insanların, bu"
3934204,3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0,33.3,"söyleyeceklerim, alakalı, danimarkayla, şimdilik","kadar, bu"
3934205,3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0,11.1,"beğenmeyi, unutmayın, kanalıma, hoşunuza, vide...",ve


In [23]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU,28.6,"staff, dizinin, betimlemesi, fi, lm","tarafından, bu"
1,4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU,42.9,"hat, sinir, katsayım, safhada","an, şu, bak"
2,5,14.213,17.219,kelimelerini lütfen seçerek konuş lütfen,fI1BI4d5KFU,20.0,"kelimelerini, konuş, seçerek",lütfen
3,6,18.687,19.854,otursana bir tanem,fI1BI4d5KFU,33.3,"tanem, otursana",bir
4,7,19.967,22.805,hah oturtmaya çalıştığına göre demek doğru,fI1BI4d5KFU,50.0,"çalıştığına, hah, oturtmaya","doğru, göre, demek"
...,...,...,...,...,...,...,...,...
3448811,3934200,590.325,594.700,zira bizim insanlarımız işine yaramayan ikinci...,dRssR_apbR0,40.0,"insanlarımız, işine, kıyafetleri, yaramayan, z...","bizim, el, yerine, ikinci"
3448812,3934201,594.755,599.701,ihtiyaç sahiplerine vermeyi tercih ediyorlar v...,dRssR_apbR0,50.0,"örneğidir, vermeyi, iyilik, ihtiyaç, sahipleri...","ve, bir, çok, tercih, bu, güzel"
3448813,3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0,43.7,"arazilerin, videoyu, danimarka, bitirmeden, yo...","bir, isterim, da, şunu, ki, olduğu, söylemek"
3448814,3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0,30.0,"bisikletle, yakını, ulaşımda, işine, yarısına,...","gidiyor, insanların, bu"


In [24]:
# # 1 way (use more memory)
# # import os
# # from multiprocessing import Process, Manager, Pool, Queue
# manager = multiprocessing.Manager()
# result_list4 = manager.list()
# index_list = manager.list()
# all_index_list = manager.list()
# var_index_list = manager.list()
# text_all_list = manager.list()  
# df = df_adjust_sentence_ratio
# df.reset_index(drop=True, inplace=True)
# index_num = list(range(len(df)))
# sent_len = word_group_adjust_select
# sent_len_num = sentence_word_count_number 
# 
# def df_exract_word_group(index_num):    
#     source_text = df.loc[index_num,"sentence"]
#     opt_var = df.loc[index_num,"video_id"]
#     words = re.findall(r"\w+", source_text, re.UNICODE)
#     # check words in word_list
#     index_list = []
#     for j in range(len(words)):
#         if words[j] in word_list:
#             index_list.append(j)
#         else:
#             pass
#     # add index plus 1 and compare with current and next index (Is it continuous or not?)
#     all_index_list = []
#     var_index_list = []
#     for k in range(len(index_list)):
#         try:
#             var1 = index_list[k] + 1  
#             var2 = index_list[k+1]
#         except:
#             var1 = index_list[k] + 1  
#             var2 = index_list[-1]
#         if var1 == var2:
#             var3 = index_list[k]
#             var_index_list.append(var3)
#         else:
#             var3 = index_list[k]
#             var_index_list.append(var3)
#             var4 = list(var_index_list)
#             all_index_list.append(var4)
#             var_index_list = []
#     # index convert to word
#     text_all_list = []
#     for m in all_index_list:
#         text_list = [] 
#         for n in m:
#             word = words[n]
#             text_list.append(word)
#             if sent_len:
#                 if len(text_list) >= sent_len_num:
#                     text = " ".join(text_list)
#                     text_all_list.append(text)
#                 else:
#                     pass
#             else:
#                text = " ".join(text_list)
#                text_all_list.append(text) 
#     # result list    
#     for search_string in text_all_list:
#         #df_var["index"] = i
#         #df_var["search_string"] = search_string
#         #df_var["sentence"] = source_text
#         #df_var["video_id"] = opt_var
#         result_list4.append([index_num,search_string,source_text,opt_var])
# 
# if __name__ == '__main__':
#     # with Pool(16) as p:
#     with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
#         p.map(df_exract_word_group, index_num)

In [25]:
# df_adjust_sentence_ratio = pd.DataFrame(list(result_list4), columns=["index","search_string","sentence","video_id"])
# df_adjust_sentence_ratio.sort_values(by="index", ascending=True, inplace=True)
# df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
# df_adjust_sentence_ratio.drop(["index"], axis=1, inplace=True)
# df_adjust_sentence_ratio

In [26]:
# 2 way
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

<ipython-input-26-ea0aea4442a6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
20,25,81.538,84.046,demek o yüzden öyle garip davrandın,fI1BI4d5KFU,83.3,davrandın,"demek, garip, o, öyle, yüzden",demek o yüzden öyle garip
32,38,135.778,139.810,o deli kadın herkesin içinde beni rezil etti h...,fI1BI4d5KFU,70.0,rezil,"içinde, beni, o, herkesin, deli, etti, kadın",o deli kadın herkesin içinde beni
37,43,147.019,149.009,ama yapma böyle git dedim git karşımdan,fI1BI4d5KFU,71.4,karşımdan,"böyle, ama, dedim, yapma, git",ama yapma böyle git dedim git
64,79,302.034,303.874,bana o kadar iyi geldi ki,fI1BI4d5KFU,100.0,NaN,"bana, kadar, o, ki, geldi, iyi",bana o kadar iyi geldi ki
67,83,311.418,313.642,oh oh yok yok yok,fI1BI4d5KFU,40.0,NaN,"oh, yok",oh oh yok yok yok
...,...,...,...,...,...,...,...,...,...
3448753,3934138,187.205,192.549,hatta köyleri o kadar sakin ki dışarıda insan ...,dRssR_apbR0,66.7,"geliyorsunuz, köyleri, görünce, sevinir","kadar, hatta, o, dışarıda, ki, insan, hale, sakin",o kadar sakin ki dışarıda insan
3448808,3934197,578.076,581.389,yani orada ciddi ciddi ikinci el kıyafet sektö...,dRssR_apbR0,66.7,"kıyafet, sektörü","orada, el, ciddi, var, yani, ikinci",yani orada ciddi ciddi ikinci el
3448810,3934199,583.933,590.183,fakat ülkemizde ikinci el kıyafet dükkanı görd...,dRssR_apbR0,60.0,"kıyafet, dükkanı, yapmıyordur, ülkemizde, görd...","iş, eğer, fakat, da, çok, el, varsa, bence, ik...",eğer varsa da bence çok iş
3448812,3934201,594.755,599.701,ihtiyaç sahiplerine vermeyi tercih ediyorlar v...,dRssR_apbR0,50.0,"örneğidir, vermeyi, iyilik, ihtiyaç, sahipleri...","ve, bir, çok, tercih, bu, güzel",ve bu çok güzel bir


In [27]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,demek o yüzden öyle garip,81.538000,83.401086,demek o yüzden öyle garip davrandın,fI1BI4d5KFU
1,o deli kadın herkesin içinde beni,135.778000,138.062800,o deli kadın herkesin içinde beni rezil etti h...,fI1BI4d5KFU
2,ama yapma böyle git dedim git,147.019000,148.549769,ama yapma böyle git dedim git karşımdan,fI1BI4d5KFU
3,bana o kadar iyi geldi ki,302.034000,303.874000,bana o kadar iyi geldi ki,fI1BI4d5KFU
4,oh oh yok yok yok,311.418000,313.642000,oh oh yok yok yok,fI1BI4d5KFU
...,...,...,...,...,...
276077,o kadar sakin ki dışarıda insan,188.084392,190.316696,hatta köyleri o kadar sakin ki dışarıda insan ...,dRssR_apbR0
276078,yani orada ciddi ciddi ikinci el,578.076000,580.178481,yani orada ciddi ciddi ikinci el kıyafet sektö...,dRssR_apbR0
276079,eğer varsa da bence çok iş,587.861571,589.528238,fakat ülkemizde ikinci el kıyafet dükkanı görd...,dRssR_apbR0
276080,ve bu çok güzel bir,597.441716,598.724012,ihtiyaç sahiplerine vermeyi tercih ediyorlar v...,dRssR_apbR0


In [28]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,demek o yüzden öyle garip,81.238000,83.701086,demek o yüzden öyle garip davrandın,fI1BI4d5KFU
1,o deli kadın herkesin içinde beni,135.478000,138.362800,o deli kadın herkesin içinde beni rezil etti h...,fI1BI4d5KFU
2,ama yapma böyle git dedim git,146.719000,148.849769,ama yapma böyle git dedim git karşımdan,fI1BI4d5KFU
3,bana o kadar iyi geldi ki,301.734000,304.174000,bana o kadar iyi geldi ki,fI1BI4d5KFU
4,oh oh yok yok yok,311.118000,313.942000,oh oh yok yok yok,fI1BI4d5KFU
...,...,...,...,...,...
276077,o kadar sakin ki dışarıda insan,187.784392,190.616696,hatta köyleri o kadar sakin ki dışarıda insan ...,dRssR_apbR0
276078,yani orada ciddi ciddi ikinci el,577.776000,580.478481,yani orada ciddi ciddi ikinci el kıyafet sektö...,dRssR_apbR0
276079,eğer varsa da bence çok iş,587.561571,589.828238,fakat ülkemizde ikinci el kıyafet dükkanı görd...,dRssR_apbR0
276080,ve bu çok güzel bir,597.141716,599.024012,ihtiyaç sahiplerine vermeyi tercih ediyorlar v...,dRssR_apbR0


In [29]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,demek o yüzden öyle garip,81,84,demek o yüzden öyle garip davrandın,fI1BI4d5KFU
1,o deli kadın herkesin içinde beni,135,138,o deli kadın herkesin içinde beni rezil etti h...,fI1BI4d5KFU
2,ama yapma böyle git dedim git,147,149,ama yapma böyle git dedim git karşımdan,fI1BI4d5KFU
3,bana o kadar iyi geldi ki,302,304,bana o kadar iyi geldi ki,fI1BI4d5KFU
4,oh oh yok yok yok,311,314,oh oh yok yok yok,fI1BI4d5KFU
...,...,...,...,...,...
276077,o kadar sakin ki dışarıda insan,188,191,hatta köyleri o kadar sakin ki dışarıda insan ...,dRssR_apbR0
276078,yani orada ciddi ciddi ikinci el,578,580,yani orada ciddi ciddi ikinci el kıyafet sektö...,dRssR_apbR0
276079,eğer varsa da bence çok iş,588,590,fakat ülkemizde ikinci el kıyafet dükkanı görd...,dRssR_apbR0
276080,ve bu çok güzel bir,597,599,ihtiyaç sahiplerine vermeyi tercih ediyorlar v...,dRssR_apbR0


In [30]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4
1,gidelim bir bakalım hep birlikte bakalım bir o...,987,988,yerleştirmeye yumurtasını seviyoruz çok güzel ...,eqkDognLtTc
2,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI
3,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw
4,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8
...,...,...,...,...,...
276077,o o o o o,407,438,o o o o o,rKpNLCUBvks
276078,s s s s s,213,215,s s s s s,Jup8NeCvwHM
276079,i i i i i,117,119,i i i i i,z8rQdsakhBM
276080,o o o o o,179,182,o o o o o,Jup8NeCvwHM


In [31]:
df_word_group_time_loc_in_sent_result["unique_word_ratio"] = df_word_group_time_loc_in_sent_result["search_string"].apply(lambda text: repetation_ratio(text))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4,93
1,gidelim bir bakalım hep birlikte bakalım bir o...,987,988,yerleştirmeye yumurtasını seviyoruz çok güzel ...,eqkDognLtTc,77
2,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI,100
3,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw,94
4,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8,89
...,...,...,...,...,...,...
276077,o o o o o,407,438,o o o o o,rKpNLCUBvks,20
276078,s s s s s,213,215,s s s s s,Jup8NeCvwHM,20
276079,i i i i i,117,119,i i i i i,z8rQdsakhBM,20
276080,o o o o o,179,182,o o o o o,Jup8NeCvwHM,20


In [32]:
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result[df_word_group_time_loc_in_sent_result["unique_word_ratio"] > unique_word_ratio]
df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-32-d6186b820154>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
<ipython-input-32-d6186b820154>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4
1,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI
2,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw
3,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8
4,sıcak mı evet böyle işte tamam bir şey yok ora...,603,612,sıcak mı evet böyle işte tamam bir şey yok ora...,Mh3bgDLKt2o
...,...,...,...,...,...
235750,ki o ya e bak,235,236,i yi de dışarı çıkmadı ki o ya e bak,gvoKkcw61pA
235751,bir de o mu i,4254,4255,nereye çıkıyor bir de o mu i şten,Yr2ZLfWT0i8
235752,o iş ben de i,1374,1375,o iş ben de i yi hadi göreyim seni,dpakDRRYQPY
235753,i ki ya da üç,142,143,i ki ya da üç görüşmeden çağrıldım çünkü,yfVrFUHoy9o


In [33]:
df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-33-e137816795b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)


,search_string,start_time,end_time,sentence,video_id,include_string_ratio
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4,100
1,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI,92
2,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw,100
3,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8,100
4,sıcak mı evet böyle işte tamam bir şey yok ora...,603,612,sıcak mı evet böyle işte tamam bir şey yok ora...,Mh3bgDLKt2o,100
...,...,...,...,...,...,...
235750,ki o ya e bak,235,236,i yi de dışarı çıkmadı ki o ya e bak,gvoKkcw61pA,36
235751,bir de o mu i,4254,4255,nereye çıkıyor bir de o mu i şten,Yr2ZLfWT0i8,39
235752,o iş ben de i,1374,1375,o iş ben de i yi hadi göreyim seni,dpakDRRYQPY,38
235753,i ki ya da üç,142,143,i ki ya da üç görüşmeden çağrıldım çünkü,yfVrFUHoy9o,32


In [34]:
df_word_group_time_loc_include_string_ratio = df_word_group_time_loc_in_sent_result_repeat_ratio[df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] > include_wordgroup_ratio]
df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)
df_word_group_time_loc_include_string_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_include_string_ratio

<ipython-input-34-ea709f90da2b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4
1,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI
2,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw
3,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8
4,sıcak mı evet böyle işte tamam bir şey yok ora...,603,612,sıcak mı evet böyle işte tamam bir şey yok ora...,Mh3bgDLKt2o
...,...,...,...,...,...
55888,bu ay mı yok mu,199,263,bu ay mı yok mu,d4hmTrjuAk0
55889,ne o boş ver ya,4366,4369,ne o boş ver ya,BeCS4X5oyXQ
55890,ya bir de o var,901,903,ya bir de o var,OfRfGDFKKio
55891,i ki üç gün mü,2912,2914,i ki üç gün mü,n5eIuJEEdgs


In [35]:
df_youtube_word_usage = word_usage_result(word_list, df_word_group_time_loc_include_string_ratio, "search_string", "video_id", word_usage_min, word_usage_max)
df_youtube_word_usage.drop_duplicates(inplace=True)
df_youtube_word_usage

,search_string,video_id
0,bunun neden böyle olduğundan emin değilim,zxznlmnXsoo
1,o yüzden yardım etmek lazım ona birkaç gün burada,ztZMp7dpRto
2,senin gibi birinin kızı olduğuna inanamıyorum,ztZMp7dpRto
3,gerek yok çok teşekkür ederim ben zaten birkaç...,zpfhmsZB26w
4,oğlum dur bırak lan bırak kızı,zoN3xOGVjbk
...,...,...
3027,peki birisi için mi yaptın yoksa oyun olsun di...,0AOYUEuSiyw
3028,gelir misin buraya sakin ol,071PtpSHv5c
3029,anne niye öyle söylüyorsun herkes yapıyor artık,071PtpSHv5c
3030,tamam merak etme buradayım canım tamam geldim,05PK5asBrP0


In [36]:
df_youtube_word_usage_merge = pd.merge(df_word_group_time_loc_include_string_ratio,df_youtube_word_usage,how="inner",on=["search_string","video_id"])
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","video_id"],inplace=True)
df_youtube_word_usage_merge.reset_index(drop=True, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4
1,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI
2,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw
3,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8
4,sıcak mı evet böyle işte tamam bir şey yok ora...,603,612,sıcak mı evet böyle işte tamam bir şey yok ora...,Mh3bgDLKt2o
...,...,...,...,...,...
3021,bi sakin ol gel otur,199,202,bi sakin ol gel otur,LnoGaqALRvI
3022,iki üç dört beş altı,468,477,iki üç dört beş altı,7ueyRTf46No
3023,on beş sene oldu be,401,403,on beş sene oldu be,vK7fxrBDlL0
3024,i pek gibi oldum be,2091,2094,i pek gibi oldum be,RCJjWkmMdD4


In [37]:
df_youtube_word_usage_merge["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id,video_url
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4,https://www.youtube.com/watch?v=KsZ_8oIQsv4&t=...
1,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI,https://www.youtube.com/watch?v=N4YDaU51lvI&t=...
2,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw,https://www.youtube.com/watch?v=CIQ3Ave21Kw&t=...
3,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8,https://www.youtube.com/watch?v=cUAQrHbbpZ8&t=...
4,sıcak mı evet böyle işte tamam bir şey yok ora...,603,612,sıcak mı evet böyle işte tamam bir şey yok ora...,Mh3bgDLKt2o,https://www.youtube.com/watch?v=Mh3bgDLKt2o&t=...
...,...,...,...,...,...,...
3021,bi sakin ol gel otur,199,202,bi sakin ol gel otur,LnoGaqALRvI,https://www.youtube.com/watch?v=LnoGaqALRvI&t=...
3022,iki üç dört beş altı,468,477,iki üç dört beş altı,7ueyRTf46No,https://www.youtube.com/watch?v=7ueyRTf46No&t=...
3023,on beş sene oldu be,401,403,on beş sene oldu be,vK7fxrBDlL0,https://www.youtube.com/watch?v=vK7fxrBDlL0&t=...
3024,i pek gibi oldum be,2091,2094,i pek gibi oldum be,RCJjWkmMdD4,https://www.youtube.com/watch?v=RCJjWkmMdD4&t=...


In [ ]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(2000)
df_youtube_word_usage_merge.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.csv", index=False)

In [ ]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(700)
df_youtube_word_usage_merge.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

In [ ]:
#df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)
#df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

In [ ]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass